# HWRS 528 IMP04
## Optimization of performance by automated calibration of parameters
### 1. Import Python packages

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import LRC_modelSubprograms as lrc      # Leaf River Catchment model subprograms
from scipy.optimize import minimize     # Optimization package for par optimization
import datetime as dt

### 2. Begin model simulation for Leaf River
#### 2.1 Load data, choose time period, and specify model parameters

In [4]:
#==== Load Leaf River Catchment Data as a DataFrame ===============================
LRData = pd.read_csv('../LeafRiverDaily.csv')

#==== Get a number of datapoints ==================================================
Ndata = LRData.shape[0]
Nvars = LRData.shape[1]

#==== Choose a simulation time period and set up time vector ======================
Period_Start = 0; Period_End = 365*3                # select time period to work with
Time = np.arange(0, Period_End, 1)
NTime = len(Time)

#==== Set number of initial time steps to use as spin-up ==========================
# Metrics are not computed on this period
SpinUp = 90                                         # select 90-day spin-up period

#==== Get data for the desired period =============================================
PPobs = LRData.PP[Time].to_numpy()                  # Convert from DataFrame to numpy
PEobs = LRData.PE[Time].to_numpy()                  # arrays to avoid indexing issues
QQobs = LRData.QQ[Time].to_numpy()

#==== Determine max values for each variable ======================================
PPobs_max = np.max(PPobs)
PEobs_max = np.max(PEobs)
QQobs_max = np.max(QQobs)

#==== Specify model parameters ====================================================
NRes = 2; ResIdex = np.arange(0, NRes, 1)

#==== Specify process parameter values for simulation==============================
Theta_C1  = 51.0;    # Upper Soil Zone Capacity
Theta_P1  = 0.75;     # Evapotranspiration Parameter (Reducing value increases ET)  
Theta_K12 = 0.25;    # Drainage to Lower Soil Zone (Increasing value increases Drainage) 
Theta_K13 = 0.04;    # Interflow Parameter (Increasing value increases Interflow)  
Theta_K2Q = 0.0002;  # Baseflow Parameter (Increasing value makes baseflow recessions steeper) 
Theta_K3Q = 0.56;     # Channel Routing Parameter (Increasing value reduces temporal dispersion)

Pars = np.zeros(6)

Pars[0] = Theta_C1;  Pars[1] = Theta_P1;  Pars[2] = Theta_K12
Pars[3] = Theta_K13; Pars[4] = Theta_K2Q; Pars[5] = Theta_K3Q

#### 2.2 Run catchment model for Selected time period

In [5]:
lrc.MainCatchModel(Pars, PPobs, PEobs, QQobs, NRes)

1.1373309606828417

### 3. Optimize values

In [7]:
#==== Set number of iterations for optimization =========================
NIterations = 10000

#==== Set paramter bounds ===============================================
            # Theta_C1  Theta_P1  Theta_K12  Theta_K13   Theta_K2Q    Theta_K3Q
ParBounds = [[10,200], [0.5,1.5], [0.0,0.5], [0.0,0.9], [0.0001,0.1], [0.1,0.9]]

#==== Perform optimization search =======================================
result = minimize(lrc.MainCatchModel, Pars, args = (PPobs, PEobs, QQobs, NRes),
                  bounds=ParBounds, method='nelder-mead',
                  options={'disp':True, 'return_all':True,'maxiter':NIterations}
                  )


Optimization terminated successfully.
         Current function value: 1.104930
         Iterations: 1782
         Function evaluations: 2700


In [21]:
#==== Save results from 2023-04-24 12:34 ================================
# Current function value: 1.104930
# result['x']
results = [7.71076000e+01, 5.00000009e-01, 4.90073149e-02, 1.87564529e-02, 1.00000000e-04, 5.76096385e-01]